## 데이터 스케일링

https://datascienceschool.net/view-notebook/f43be7d6515b48c0beb909826993c856/

### IN 파일
datas 폴더내 위치 (전처리 쥬피터 파일을 통해 생성)
1. raw_all_preprocessing.csv
2. raw_main_preprocessing.csv

### OUT 파일
datas 폴더에 생성됨
1. R_train.csv , R_train_main.csv, R_test.csv , R_test_main.csv : R분석용으로 분리된 데이터셋
2. keep_R_dataset : R분석용(예측결과 건 체크용)

### 주의! 
해당작업에서 예외적으로 '_x, _y 변수/파일명의 의미'는 '정상' ,'부실'의 의미로 쓰임

(* 일반적으로 _y는 label을 의미)

## 사전값 셋팅 필수

In [30]:
## main용인지 all용인지 미리 셋팅
df = 'main'
# df = 'all'

In [31]:
import pandas as pd
import numpy as np

## 전처리된 데이터 가져오기

In [32]:
if df == 'all':
    address = './datas/raw_all_preprocessing.csv'
else:
    address = './datas/raw_main_preprocessing.csv'

data = pd.read_csv(address, header=0)  
cols = data.columns.values
cols = cols[1:].tolist()  # 주의! 


# 해당연도 추출
data['year'] = data.key.map(lambda x : str(x)[0:4]) 

# 문자열 합치기
data['split_key'] = data[['year', 'industry']].apply(lambda x: ''.join(x.astype(str)), axis=1)
af_cols = data.columns.values

# # 스케일 조정할 factor들만 셋팅
# data = data.reindex(cols,axis = 1)
data.head()


,Unnamed: 0,key,industry,label,6000903001D1,6000902001D1,6000909001D1,6000904001D3,6000901001D1,6000908001O0,...,DR00000162,DR00000056,DR00000110,DR00000164,S410001300D1,S41000210FD2,S430000600D2,S410001300D2,year,split_key
0,0,2013000020,C,0,0.030707,1.074354,-0.156980,2.0,21.785946,22.127060,...,0.0,0.0,0.0,0.0,2.254586,-0.688627,2.663963,0.507419,2013,2013C
1,1,2013000040,C,0,2.099808,0.369686,-0.289442,0.0,21.725441,22.125185,...,0.0,0.0,0.0,0.0,2.254586,-0.394038,2.663963,0.420925,2013,2013C
2,2,2013000050,C,0,2.099808,0.367136,0.220773,0.0,21.918195,22.130386,...,0.0,0.0,0.0,0.0,2.254586,0.401272,2.663963,0.420925,2013,2013C
3,3,2013000070,M,0,2.099808,1.074354,-0.368501,0.0,22.187700,22.120351,...,0.0,0.0,1.0,1.0,0.123879,0.110957,2.663963,0.334778,2013,2013M
4,4,2013000080,C,0,2.099808,0.837905,-0.065866,1.0,22.121984,22.146062,...,1.0,1.0,1.0,1.0,0.702868,-0.556720,2.978547,0.774888,2013,2013C


## 데이터셋 분리하기

In [33]:
# 데이터셋 분리

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# 연도별 산업별 (split_key) 기준 분리
temp = data[data.label == 0]
distnct_key = temp['split_key'].drop_duplicates()



# 1) 정상
result_test = pd.DataFrame()
result_train = pd.DataFrame()

# 연도별 산업별 데이터 루프
for x in distnct_key:
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    temp = data.query("(split_key == '" + x + "') and (label == 0)")
    X = temp.iloc[:,:].values
    X_train, X_test, = train_test_split(X, 
                                            test_size=0.3, # train에 복제 데이터 추가될 예정이므로 0.3
                                            random_state=42) 

    train_df = pd.DataFrame(X_train)
    test_df = pd.DataFrame(X_test)
    result_train = [result_train, train_df]
    result_train = pd.concat(result_train) 
    result_test = [result_test, test_df]
    result_test = pd.concat(result_test)


# 2) 부실
result_test_y = pd.DataFrame()
result_train_y = pd.DataFrame()

# 산업별 분리는 무리, 연도별 적용
temp = data[data.label == 1]
distnct_key = temp['year'].drop_duplicates()


for x in distnct_key:
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    temp = data.query("(year == '" + x + "') and (label == 1)")


    Y = temp.iloc[:,:].values
    Y_train, Y_test, = train_test_split(Y, 
                                            test_size=0.3, # train에 복제 데이터 추가될 예정이므로 0.3
                                            random_state=42) 


    train_df = pd.DataFrame(Y_train)
    test_df = pd.DataFrame(Y_test)
    result_train_y = [result_train_y, train_df]
    result_train_y = pd.concat(result_train_y) 
    result_test_y = [result_test_y, test_df]
    result_test_y = pd.concat(result_test_y)
    


    
    

In [34]:
result_train_y.head()

,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,691,2013036210,C,1,2.09981,3.63332,-0.589807,3,21.6912,22.1107,...,3,0,0,0,2.25459,-0.312475,2.66396,0.420925,2013,2013C
1,281,2013007390,C,1,2.09981,0.472573,-0.225001,0,21.7082,22.1259,...,3,3,3,3,2.25459,0.891222,2.66396,0.420925,2013,2013C
2,722,2013037950,C,1,2.09981,0.0865908,-0.426403,2,21.7211,22.1264,...,3,1,2,2,2.25459,-0.830058,2.66396,0.420925,2013,2013C
3,754,2013039850,G,1,2.09981,1.07435,-1.98553,2,21.7071,22.1251,...,3,0,1,3,2.25459,-1.34481,2.66396,0.420925,2013,2013G
4,775,2013041960,C,1,2.09981,0.736851,-0.146635,0,21.7208,22.1268,...,0,0,0,0,2.25459,-0.183044,2.66396,0.207801,2013,2013C


In [35]:
len(data), len(result_train)+len(result_test), len(data[data.label == 0])

(7626, 7191, 7191)

In [36]:
len(data), len(result_train_y)+len(result_test_y), len(data[data.label == 1])

(7626, 435, 435)

In [37]:
len(result_train_y),len(result_test_y)

(302, 133)

In [38]:
#  키별로 비율 배분이 잘 됐는지 확인
if df == 'all':
    max_idx = 707
else:
    max_idx = 123
    
train_x_pivot = pd.pivot_table(result_train, index = [max_idx], values = [0], aggfunc = [len] )
test_x_pivot = pd.pivot_table(result_test, index = [max_idx], values = [0], aggfunc = [len] )
x_pivot = pd.merge(train_x_pivot, test_x_pivot, left_index=True, right_index=True)
x_pivot

,len_x,len_y
,0,0
123,,
2013A,2,2
2013C,609,262
2013D,7,3
2013E,2,1
2013F,29,13
2013G,68,30
2013H,14,6
2013J,93,41


## 데이터 셔플하기

In [39]:
# 데이터 셔플
from sklearn.utils import shuffle

for x in range(0,100):
    result_train= shuffle(result_train, random_state=42)
    result_test = shuffle(result_test, random_state=42)
    result_train_y = shuffle(result_train_y, random_state=42)
    result_test_y = shuffle(result_test_y, random_state=42)

# 인덱스 리셋
result_train = result_train.reset_index()
result_test = result_test.reset_index()
result_train_y = result_train_y.reset_index()
result_test_y = result_test_y.reset_index()

In [40]:
# TrainSet
train_set = pd.DataFrame()
train_set = pd.concat([result_train, result_train_y])   # 정상, 부실 기업 통합
# 인덱스 리셋
train_set = train_set.reset_index()
train_set = train_set.drop(['level_0','index', 0, max_idx-1,max_idx],axis=1)  # 열삭제

# 인덱스명 새로 지정
train_set.columns = af_cols[1:-2]



# TestSet
test_set = pd.DataFrame()
test_set = pd.concat([result_test, result_test_y])   # 정상, 부실 기업 통합
# 인덱스 리셋
test_set = test_set.reset_index()
test_set = test_set.drop(['level_0','index', 0, max_idx-1,max_idx],axis=1)  # 열삭제

# 인덱스명 새로 지정
test_set.columns = af_cols[1:-2]

In [41]:
# 변수저장
import pickle # 파이썬 객체저장을 위한 모듈

if df == 'all':
    file_nm = './pickles/dataset_all.p'
else:
    file_nm = './pickles/dataset_main.p'
    
# 파이썬 객체 상태로 저장하기
with open(file_nm, 'wb') as file:  # hello.txt 파일을 바이너리 쓰기 모드(wb)
    pickle.dump(train_set, file)
    pickle.dump(test_set, file)
    pickle.dump(af_cols, file)


In [42]:
# 정상 : 부실 (6:4로 맞추기 위함) 임시 데이터셋 생성
temp_train = result_train[:453]
temp_test = result_test[:200] 
temp_train_set = pd.concat([temp_train, result_train_y])   # 정상, 부실 기업 통합
temp_test_set = pd.concat([temp_test, result_test_y])   # 정상, 부실 기업 통합

print(temp_train_set.shape,temp_test_set.shape )

(755, 125) (333, 125)


In [43]:
# 변수저장
import pickle # 파이썬 객체저장을 위한 모듈

if df == 'all':
    file_nm = './pickles/data6vs4_all.p'
else:
    file_nm = './pickles/data6vs4_main.p'
    
# 파이썬 객체 상태로 저장하기
with open(file_nm, 'wb') as file:  # hello.txt 파일을 바이너리 쓰기 모드(wb)
    pickle.dump(temp_train_set, file)
    pickle.dump(temp_test_set, file)
    pickle.dump(af_cols, file)

In [44]:
temp_train_set.head()

,index,0,1,2,3,4,5,6,7,8,...,114,115,116,117,118,119,120,121,122,123
0,89,7578,2017217190,C,0,2.09981,0.270822,-0.500636,0,21.7127,...,0,0,0,1,2.25459,-2.21916,2.66396,0.420925,2017,2017C
1,468,5664,2016097870,C,0,0.827262,1.07435,0.772678,1,21.7263,...,0,0,0,0,2.25459,-1.13276,2.66396,0.420925,2016,2016C
2,123,4252,2015115610,C,0,1.0853,1.07435,-0.199992,1,21.7158,...,0,0,0,1,2.25459,1.15384,2.66396,0.420925,2015,2015C
3,339,7193,2017093050,C,0,2.09981,1.07435,0.190371,0,22.0504,...,0,0,0,0,2.40525,1.04229,0.267563,2.26534,2017,2017C
4,0,207,2013005030,C,0,2.09981,1.75398,-0.23022,1,21.7209,...,1,0,0,1,2.25459,-0.61853,2.66396,0.420925,2013,2013C


### R 분석용 백업 파일 생성

In [45]:
# 반복 문자 리스트 생성 함수
def str_repeat(str,n):
    str_list = []
    
    for x in range(n):
        str_list.append(str)
    
    return str_list
    

In [46]:
keep_R_train = pd.DataFrame()
temp_str = str_repeat('train', len(temp_train_set))
keep_R_train['df'] = temp_str
keep_R_train['year'] = list(temp_train_set[max_idx-1])
keep_R_train['industry'] = list(temp_train_set[2])
keep_R_train['code'] = list(temp_train_set[1].map(lambda x : str(x)[4:]))
keep_R_train['label'] = list(temp_train_set[3])

keep_R_test = pd.DataFrame()
temp_str = str_repeat('test', len(temp_test_set))
keep_R_test['df'] = temp_str
keep_R_test['year'] = list(temp_test_set[max_idx-1])
keep_R_test['industry'] = list(temp_test_set[2])
keep_R_test['code'] = list(temp_test_set[1].map(lambda x : str(x)[4:]))
keep_R_test['label'] = list(temp_test_set[3])


keep_R_dataset = pd.concat([keep_R_train,keep_R_test])
keep_R_dataset.head()
# keep_R_dataset.tail()


,df,year,industry,code,label
0,train,2017,C,217190,0
1,train,2016,C,097870,0
2,train,2015,C,115610,0
3,train,2017,C,093050,0
4,train,2013,C,005030,0


In [47]:
# 파일 저장
keep_R_dataset.to_csv('./datas/keep_R_dataset.csv', index=False)

### R 분석용 train, test set 생성

In [48]:
R_train_y = temp_train_set[3]
R_train = temp_train_set.drop(['index', 0, 1, 2, 3, max_idx-1,max_idx],axis=1)  # 열삭제
R_test_y = temp_test_set[3]
R_test = temp_test_set.drop(['index', 0, 1, 2, 3, max_idx-1,max_idx],axis=1)  # 열삭제

In [49]:
# 인덱스명 새로 지정
R_train.columns = cols[3:]
R_test.columns = cols[3:]

R_train['label'] = R_train_y
R_test['label'] = R_test_y

In [50]:
# 파일 저장
if df == 'all':
    R_train.to_csv('./datas/R_train.csv', index=False)
    R_test.to_csv('./datas/R_test.csv', index=False)
    print('all 데이터 생성')
else:
    R_train.to_csv('./datas/R_train_main.csv', index=False)
    R_test.to_csv('./datas/R_test_main.csv', index=False)  
    print('main 데이터 생성')
    

main 데이터 생성


In [51]:
df

'main'

# 아래는 작업 전!!! 

## 데이터 스케일링(0~255)

In [60]:
# 정상 부실 기업 합치기
train_set = pd.concat([result_train,result_train_y], axis=1)
test_set = pd.concat([result_test,result_test_y], axis=1)

print(train_set.shape, test_set.shape)

(4988, 1418) (2203, 1417)


### 참고! 데이터를 붙였다 떼었다를 반복하는 이유!
1. 분리 : 데이터셋에서 정상 부실간에 연도별/산업별 비율을 맞춘다.
2. 합체 : 트레이닝셋을 기준으로 스케일링 한 후 테스트셋도 스케일링 한다.
3. 분리 : 트레이닝셋의 정상기업수가 매우 많으므로 일부만 사용할 예정(부실기업은 전체를 다 사용한다.)

단 0~255 스케일 조정은 나중에 함 -> R을 통한 변수추출 및 로지스틱 적용시엔 방향확인이 필요함

In [17]:
# 확인

In [18]:
print(len(X_train),X_train[0].min(),X_train[0].max())

1388 0.0 241.57894736842104


In [19]:
print(len(X_train),X_train[-1].min(),X_train[-1].max())

1388 0.0 255.0


In [20]:
print(len(X_train),X_train.min(),X_train.max())

1388 0.0 255.0


In [21]:
print(len(X_test),X_test.min(),X_test.max())

596 0.0 255.0
